In [1]:
import torch

print(torch.__version__)

2.0.1+cu117


In [34]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision
from torchvision import transforms
import csv
from torch.nn.utils.rnn import pad_sequence

In [2]:
img_dir='/home/irteam/junghye-dcloud-dir/Pixt/data/dataset/dataset1'
label_dir='/home/irteam/junghye-dcloud-dir/Pixt/data/dataset/dataset1/motion_excluded.csv'

In [35]:
#sol2

class CustomDataset(Dataset):
  def __init__(self,img_dir:str,label_dir:str,transform):
  
    super().__init__
    self.img_dir=img_dir
    self.images=os.listdir(self.img_dir)
    self.transform=transform
    all_labels=[]
    with open(label_dir,'r') as f:
      data=csv.reader(f)
      self.label_data=list(data)
      # 전체 label
      for row in self.label_data:
        all_labels.extend(row[2:])

      unique_labels=list(set(all_labels))

    self.label_to_idx={unique_labels[i]:i+1 for i in range(len(unique_labels))}
    


  def __len__(self):
    return (len(self.images))

  def __getitem__(self,idx):
    file_path=os.path.join(self.img_dir,self.images[idx])
    img=Image.open(file_path).convert('RGB')
    img_transformed=self.transform(img)

    target_row=self.label_data[idx]
    target_list=[self.label_to_idx[item] for item in target_row[2:]]
      
    target_tensor=torch.tensor(target_list).float()


    return img_transformed,target_tensor
   

In [36]:
train_transform=transforms.Compose([
    transforms.Resize((256, 256)),          # 개와 고양이 사진 파일의 크기가 다르므로, Resize로 맞춰줍니다.
    transforms.CenterCrop((224, 224)),      # 중앙 Crop
    transforms.RandomHorizontalFlip(0.5),   # 50% 확률로 Horizontal Flip
    transforms.ToTensor(), 
])

In [37]:
train_dataset = CustomDataset(img_dir,label_dir,train_transform)

In [43]:
x,y=train_dataset.__getitem__(3)
print(x.shape,y.shape)

torch.Size([3, 224, 224]) torch.Size([15])


In [39]:
def collate_fn(batchDummy): 

  images_list,label_list,=[],[]

  for (_img,_label) in batchDummy:
    images_list.append(_img)
    label_list.append(_label)
  label_list=pad_sequence(label_list,batch_first=True,padding_value=0)
  return images_list,label_list
  

In [40]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,collate_fn=collate_fn)

In [42]:
batch=next(iter(train_loader))

data,labels=batch

for i in range(len(data)):
    sample_data=data[i]
    sample_label=labels[i]

    print('sample_data:',sample_data.shape)
    print('sample_label',sample_label.shape)

sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 224])
sample_label torch.Size([28])
sample_data: torch.Size([3, 224, 2